In [50]:
#data science and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

#creation of dataset
import _lib.ml_workflow.create_dataset as cds
from _lib.export import to_csv
from _lib.raman_lib.misc import load_data

#quality control
import _lib.ml_workflow.quality_control as qc
from _lib.raman_lib.preprocessing import RangeLimiter
from _lib.raman_lib.visualization import plot_spectra_peaks
from _lib.raman_lib.spectra_scoring import score_names

#preprocessing
from _lib.ml_workflow.preprocess_data import preprocess

#model creation
from sklearn.model_selection import LeaveOneGroupOut, cross_validate, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import FeatureAgglomeration
from sklearn.decomposition import PCA, NMF
from sklearn.pipeline import Pipeline
from _lib.raman_lib.preprocessing import PeakPicker

#file handling
from pathlib import Path
import os

In [51]:
# define the paths to all experiment data
# each dir contains all the files associated with that experiment
# each file has a prefix that indicates the group and the measurement --> group_measurement_.....
path_etoposide = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide"
path_resveratrol = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol"
path_both = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Both"
path_control = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Control"

# Define parameters
In order to function properly, the provided code depends on predefined parameters like output paths, limits and thresholds for the quality control, ...
## Define data paths
Define the location of the data, and where quality-controlled and preprocessed data should be stored. Both of them rely on a unique file-prefix that describes the data being analyzed.

In [52]:
FILE_PREFIX = "HL_428_E_R"
DATASET_OUT = "./data/" + FILE_PREFIX + ".csv"
RESULT_DIR = "./result/" + FILE_PREFIX
QC_OUT = RESULT_DIR + "/" + FILE_PREFIX + "_qc.csv"
PREP_OUT = RESULT_DIR + "/" + FILE_PREFIX + "_preprocessed.csv"
LDA_DIR = RESULT_DIR + "/" + "lda_dim_reduction"
REG_DIR = RESULT_DIR + "/" + "regularized_models/"
TREE_DIR = RESULT_DIR + "/" + "tree_based_models/"

## Define quality scoring parameters
The quality control only uses peaks in a given interval, recognizes peaks via a filter (Sav-Gol) and scores them based on some metrics. Finally, the best N spectra are selected.
### Spectral Range Limits

In [53]:
QC_LIM_LOW = 450
QC_LIM_HIGH = 1650

### Peak Detection

In [54]:
QC_WINDOW = 35
QC_THRESHOLD = 0.001
QC_MIN_HEIGHT = 50

### Scoring

In [55]:
QC_SCORE = 1
QC_PEAKS = 1

### Number of spectra to keep

In [56]:
QC_NUM = 300

## Define Preprocessing Parameter
### Spectral Range Limits

In [57]:
PREP_LIM_LOW = QC_LIM_LOW
PREP_LIM_HIGH = QC_LIM_HIGH

### Window-width for smoothing

In [58]:
PREP_WINDOW = 15

## Settings for Cross Validation

In [59]:
SCORING = ['roc_auc', 'accuracy', 'f1']
N_TRIALS = 20
N_FOLDS = 5
N_CORES = -1

# Create the dataset
Create the dataset using the implementation provided by D. Zimmermann.
For the creation of the dataset, the two source dirs, as well as the desired labels are needed.
Furthermore, an output directory is needed, to store the created dataset 

In [60]:
datadir = Path(DATASET_OUT).parent
if not os.path.exists(datadir):
    os.makedirs(datadir)

dataset = cds.create_dataset([path_etoposide, path_resveratrol], ['Etoposide', 'Resveratrol'], grouped=True)
dataset.to_csv(DATASET_OUT, index=False)

root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol
root - INFO - Finished loading data.
root - INFO - Finished loading data.
root - INFO - Finished loading data.


# Do quality control
Asses the spectra based on their quality, and remove low quality spectra

In [61]:
path_in = Path(DATASET_OUT)
path_out = Path(RESULT_DIR)

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out_data = path_out / (path_in.stem + "_qc.csv")
path_out_scores = path_out / (path_in.stem + "_qc_scores.csv")

data = pd.read_csv(path_in)

data_out, _, score_dict = qc.score_sort_spectra(data,
                                                n=QC_NUM,
                                                limits=[QC_LIM_LOW, QC_LIM_HIGH],
                                                bl_method="asls",
                                                sg_window=QC_WINDOW,
                                                threshold=QC_THRESHOLD,
                                                min_height=QC_MIN_HEIGHT,
                                                score_measure=QC_SCORE,
                                                n_peaks_influence=QC_PEAKS,
                                                detailed=True)

visualize = False
if visualize:
    data_vis = data.drop(columns=["label", "file", "group"]).values.astype(float)
    wns_vis = data.drop(columns=["label", "file", "group"]).columns.astype(float)

    rl = RangeLimiter(lim=[QC_LIM_LOW, QC_LIM_HIGH],
                      reference=wns_vis)

    data_rl = rl.fit_transform(data_vis)
    wns_rl = wns_vis[rl.lim_[0]:rl.lim_[1]]

    plot_spectra_peaks(wns_rl,
                       data_rl,
                       score_dict["peak_pos"],
                       labels=score_dict["total_scores"])

data_out.to_csv(path_out_data, index=False)

pd.DataFrame({score_names[QC_SCORE]: score_dict["intensity_scores"],
              "N Peaks": score_dict["peak_scores"]}).to_csv(
    path_out_scores, index=False
)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Preprocess the data

In [62]:
path_in = Path(QC_OUT)
path_out = Path(RESULT_DIR)

filename = path_in.stem.removesuffix("_qc")

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out = path_out / (filename + "_preprocessed.csv")

data = load_data(QC_OUT)

# save groups and remove column, otherwise preprocess won't work
groups = np.asarray(data.group)
data = data.drop(columns=["group"])

data_prep = preprocess(data, limits=[PREP_LIM_LOW, PREP_LIM_HIGH], sg_window=PREP_WINDOW)

#add groups again
data_prep.insert(2, "group", groups)

data_prep.to_csv(path_out, index=False)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Implement GroupKFold CV
## LDA Dimensionality Reduction 

In [63]:
path_in = PREP_OUT
#path_out = Path(args.out)

#filename = path_in.stem

data = load_data(path_in)

X = data.drop(columns=["label", "file", "group"])
wns = np.asarray(X.columns.astype(float))
X = np.asarray(X)
y = np.array(data.label)
y, y_key = pd.factorize(y)

#split the dataset according to the groups
groups = np.array(data.group)
logo = LeaveOneGroupOut()



### Baseline with LDA alone

In [64]:
clf = LinearDiscriminantAnalysis()
result = cross_validate(clf, X, y, cv=logo, scoring=SCORING, return_train_score=True, groups=groups)
to_csv(result, path=LDA_DIR + "/lda", scoring=SCORING, param_opt=False)
result

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


{'fit_time': array([0.2639029 , 0.17492414, 0.15228105]),
 'score_time': array([0.00253725, 0.00236988, 0.00228   ]),
 'test_roc_auc': array([0.73047786, 0.72267298, 0.6881299 ]),
 'train_roc_auc': array([1., 1., 1.]),
 'test_accuracy': array([0.67487685, 0.71957672, 0.67307692]),
 'train_accuracy': array([1., 1., 1.]),
 'test_f1': array([0.7027027 , 0.70056497, 0.69642857]),
 'train_f1': array([1., 1., 1.])}

### Feature Selection with PCA followed by LDA

In [65]:
param_grid = {"pca__n_components": range(
    1, 51, 10
)}
clf = Pipeline([("pca", PCA()),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy',
                       return_train_score=True, verbose=3).fit(X, y, groups=groups)

to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/pca_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)


Fitting 3 folds for each of 5 candidates, totalling 15 fits
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel

### Non-Negative Matrix Factorization

In [66]:
"""clf = Pipeline([("nmf", NMF(init="nndsvda", tol=1e-2, max_iter=5000)),
                ("lda", LinearDiscriminantAnalysis())])
cross_validate(clf, X, y, cv=logo, groups=groups, scoring=SCORING)"""

'clf = Pipeline([("nmf", NMF(init="nndsvda", tol=1e-2, max_iter=5000)),\n                ("lda", LinearDiscriminantAnalysis())])\ncross_validate(clf, X, y, cv=logo, groups=groups, scoring=SCORING)'

### Feature Agglomeration

In [67]:
param_grid = {"agglo__n_clusters": range(
    5, 41, 5
)}
clf = Pipeline([("agglo", FeatureAgglomeration(connectivity=np.diag(np.ones(len(wns))) +
                                                            np.diag(np.ones(len(wns) - 1), 1) +
                                                            np.diag(np.ones(len(wns) - 1), -1))),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy',
                       return_train_score=True, verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/fa_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV 1/3] END agglo__n_clusters=5; accuracy: (train=0.690, test=0.685) f1: (train=0.712, test=0.663) roc_auc: (train=0.772, test=0.751) total time=   0.1s
[CV 2/3] END agglo__n_clusters=5; accuracy: (train=0.735, test=0.598) f1: (train=0.754, test=0.568) roc_auc: (train=0.823, test=0.609) total time=   0.1s
[CV 3/3] END agglo__n_clusters=5; accuracy: (train=0.676, test=0.726) f1: (train=0.628, test=0.755) roc_auc: (train=0.723, test=0.733) total time=   0.1s
[CV 1/3] END agglo__n_clusters=10; accuracy: (train=0.725, test=0.709) f1: (train=0.740, test=0.674) roc_auc: (train=0.783, test=0.781) total time=   0.1s
[CV 2/3] END agglo__n_clusters=10; accuracy: (train=0.771, test=0.598) f1: (train=0.785, test=0.563) roc_auc: (train=0.845, test=0.598) total time=   0.1s
[CV 3/3] END agglo__n_clusters=10; accuracy: (train=0.686, test=0.726) f1: (train=0.668, test=0.771) roc_auc: (train=0.749, test=0.751) total time=   0.1s
[CV 1/3] END 

### PeakPicker

In [68]:
param_grid = {"peaks__min_dist": range(
    10, 151, 10
)}
clf = Pipeline([("peaks", PeakPicker()),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/peak_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)


Fitting 3 folds for each of 15 candidates, totalling 45 fits
[CV 1/3] END peaks__min_dist=10; accuracy: (train=0.773, test=0.645) f1: (train=0.789, test=0.586) roc_auc: (train=0.857, test=0.729) total time=   0.0s
[CV 2/3] END peaks__min_dist=10; accuracy: (train=0.808, test=0.646) f1: (train=0.816, test=0.621) roc_auc: (train=0.886, test=0.646) total time=   0.0s
[CV 3/3] END peaks__min_dist=10; accuracy: (train=0.796, test=0.678) f1: (train=0.783, test=0.745) roc_auc: (train=0.858, test=0.696) total time=   0.0s
[CV 1/3] END peaks__min_dist=20; accuracy: (train=0.768, test=0.655) f1: (train=0.780, test=0.598) roc_auc: (train=0.850, test=0.732) total time=   0.0s
[CV 2/3] END peaks__min_dist=20; accuracy: (train=0.805, test=0.656) f1: (train=0.814, test=0.620) roc_auc: (train=0.882, test=0.664) total time=   0.0s
[CV 3/3] END peaks__min_dist=20; accuracy: (train=0.796, test=0.678) f1: (train=0.783, test=0.745) roc_auc: (train=0.858, test=0.696) total time=   0.0s
[CV 1/3] END peaks__m

## Regularized Models
### Logistic Regression L1 Penalty

In [69]:
param_grid = {
    "logreg__C": np.logspace(-2, 1, 16)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(solver="liblinear",
                                  penalty="l1", max_iter=1000, random_state=41))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/logreg_l1", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV 1/3] END logreg__C=0.01; accuracy: (train=0.494, test=0.512) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 2/3] END logreg__C=0.01; accuracy: (train=0.482, test=0.540) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 3/3] END logreg__C=0.01; accuracy: (train=0.526, test=0.452) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 1/3] END logreg__C=0.015848931924611134; accuracy: (train=0.733, test=0.704) f1: (train=0.748, test=0.667) roc_auc: (train=0.814, test=0.805) total time=   0.0s
[CV 2/3] END logreg__C=0.015848931924611134; accuracy: (train=0.788, test=0.646) f1: (train=0.799, test=0.604) roc_auc: (train=0.863, test=0.704) total time=   0.0s
[CV 3/3] END logreg__C=0.015848931924611134; accuracy: (train=0.768, test=0.692) f1: (train=0.766, test=0.744) roc_auc: (train=0.836, test=0.696) total time=   0.

### Logistic Regression L2 Penalty

In [70]:
param_grid = {
    "logreg__C": np.logspace(-5, 1, 13)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(solver="liblinear",
                                  penalty="l2", max_iter=1000, random_state=51))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/logreg_l2", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 13 candidates, totalling 39 fits
[CV 1/3] END logreg__C=1e-05; accuracy: (train=0.746, test=0.670) f1: (train=0.761, test=0.613) roc_auc: (train=0.831, test=0.783) total time=   0.0s
[CV 2/3] END logreg__C=1e-05; accuracy: (train=0.774, test=0.630) f1: (train=0.784, test=0.593) roc_auc: (train=0.863, test=0.666) total time=   0.0s
[CV 3/3] END logreg__C=1e-05; accuracy: (train=0.724, test=0.788) f1: (train=0.711, test=0.833) roc_auc: (train=0.811, test=0.809) total time=   0.0s
[CV 1/3] END logreg__C=3.1622776601683795e-05; accuracy: (train=0.751, test=0.685) f1: (train=0.765, test=0.636) roc_auc: (train=0.848, test=0.792) total time=   0.0s
[CV 2/3] END logreg__C=3.1622776601683795e-05; accuracy: (train=0.788, test=0.630) f1: (train=0.800, test=0.588) roc_auc: (train=0.875, test=0.667) total time=   0.0s
[CV 3/3] END logreg__C=3.1622776601683795e-05; accuracy: (train=0.742, test=0.779) f1: (train=0.731, test=0.826) roc_auc: (train=0.834, test=0.814) total t

### Linear SVM L1 Penalty

In [71]:
param_grid = {
    "svm__C": np.logspace(-3, 0, 16)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(penalty="l1", dual=False, max_iter=10000))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/svm_l1", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV 1/3] END svm__C=0.001; accuracy: (train=0.494, test=0.512) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 2/3] END svm__C=0.001; accuracy: (train=0.482, test=0.540) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 3/3] END svm__C=0.001; accuracy: (train=0.526, test=0.452) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 1/3] END svm__C=0.001584893192461114; accuracy: (train=0.494, test=0.512) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 2/3] END svm__C=0.001584893192461114; accuracy: (train=0.482, test=0.540) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 3/3] END svm__C=0.001584893192461114; accuracy: (train=0.526, test=0.452) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 1/3] END

### Linear SVM L2 Penalty

In [72]:
param_grid = {
    "svm__C": np.logspace(-5, -1, 13)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(penalty="l2", max_iter=5000))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/svm_l2", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 13 candidates, totalling 39 fits
[CV 1/3] END svm__C=1e-05; accuracy: (train=0.773, test=0.714) f1: (train=0.787, test=0.659) roc_auc: (train=0.875, test=0.804) total time=   0.0s
[CV 2/3] END svm__C=1e-05; accuracy: (train=0.815, test=0.640) f1: (train=0.823, test=0.605) roc_auc: (train=0.894, test=0.678) total time=   0.0s
[CV 3/3] END svm__C=1e-05; accuracy: (train=0.773, test=0.769) f1: (train=0.766, test=0.820) roc_auc: (train=0.867, test=0.820) total time=   0.0s
[CV 1/3] END svm__C=2.1544346900318823e-05; accuracy: (train=0.811, test=0.724) f1: (train=0.819, test=0.671) roc_auc: (train=0.900, test=0.815) total time=   0.0s
[CV 2/3] END svm__C=2.1544346900318823e-05; accuracy: (train=0.835, test=0.656) f1: (train=0.842, test=0.629) roc_auc: (train=0.913, test=0.695) total time=   0.0s
[CV 3/3] END svm__C=2.1544346900318823e-05; accuracy: (train=0.814, test=0.769) f1: (train=0.810, test=0.817) roc_auc: (train=0.897, test=0.820) total time=   0.0s
[CV 1/

## Tree-based models
### Basic Decision Tree


In [73]:
param_grid = {
    "ccp_alpha": np.logspace(-3, -1, 9)
}
clf = DecisionTreeClassifier(random_state=653)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=TREE_DIR + "/decision_tree", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.675) f1: (train=1.000, test=0.616) roc_auc: (train=1.000, test=0.672) total time=   0.4s
[CV 2/3] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.577) f1: (train=1.000, test=0.535) roc_auc: (train=1.000, test=0.573) total time=   0.5s
[CV 3/3] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.587) f1: (train=1.000, test=0.613) roc_auc: (train=1.000, test=0.585) total time=   0.4s
[CV 1/3] END ccp_alpha=0.0017782794100389228; accuracy: (train=1.000, test=0.675) f1: (train=1.000, test=0.616) roc_auc: (train=1.000, test=0.672) total time=   0.4s
[CV 2/3] END ccp_alpha=0.0017782794100389228; accuracy: (train=1.000, test=0.577) f1: (train=1.000, test=0.535) roc_auc: (train=1.000, test=0.573) total time=   0.5s
[CV 3/3] END ccp_alpha=0.0017782794100389228; accuracy: (train=1.000, test=0.587) f1: (train=1.000, test=0.613) roc_auc: (train=1.000, test=0.585) total time=

### Random Forest

In [74]:
param_grid = {
    "colsample_bytree": np.linspace(0.01, 0.2, 20)
}
clf = LGBMClassifier(boosting_type="rf",
                     subsample=0.8,
                     subsample_freq=1,
                     max_bin=10,
                     max_depth=8,
                     random_state=2434)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=TREE_DIR + "/random_forest", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV 1/3] END colsample_bytree=0.01; accuracy: (train=0.874, test=0.675) f1: (train=0.877, test=0.598) roc_auc: (train=0.958, test=0.812) total time=   0.1s
[CV 2/3] END colsample_bytree=0.01; accuracy: (train=0.886, test=0.640) f1: (train=0.891, test=0.595) roc_auc: (train=0.957, test=0.697) total time=   0.1s
[CV 3/3] END colsample_bytree=0.01; accuracy: (train=0.893, test=0.755) f1: (train=0.890, test=0.798) roc_auc: (train=0.964, test=0.798) total time=   0.1s
[CV 1/3] END colsample_bytree=0.02; accuracy: (train=0.892, test=0.695) f1: (train=0.894, test=0.627) roc_auc: (train=0.964, test=0.821) total time=   0.1s
[CV 2/3] END colsample_bytree=0.02; accuracy: (train=0.881, test=0.619) f1: (train=0.885, test=0.571) roc_auc: (train=0.958, test=0.703) total time=   0.1s
[CV 3/3] END colsample_bytree=0.02; accuracy: (train=0.906, test=0.750) f1: (train=0.902, test=0.794) roc_auc: (train=0.968, test=0.780) total time=   0.2s
[CV

### Gradient-boosted Decision Tree

In [75]:
param_grid = {
    "learning_rate": np.linspace(0.01, 0.2, 20)
}
clf = LGBMClassifier(colsample_bytree=0.2,
                     max_bin=10,
                     max_depth=5,
                     random_state=6233)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=TREE_DIR + "/gbdt", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV 1/3] END learning_rate=0.01; accuracy: (train=0.977, test=0.700) f1: (train=0.978, test=0.635) roc_auc: (train=0.998, test=0.825) total time=   0.3s
[CV 2/3] END learning_rate=0.01; accuracy: (train=0.966, test=0.651) f1: (train=0.967, test=0.616) roc_auc: (train=0.991, test=0.723) total time=   0.3s
[CV 3/3] END learning_rate=0.01; accuracy: (train=0.967, test=0.731) f1: (train=0.965, test=0.780) roc_auc: (train=0.998, test=0.765) total time=   0.2s
[CV 1/3] END learning_rate=0.02; accuracy: (train=0.995, test=0.700) f1: (train=0.995, test=0.621) roc_auc: (train=1.000, test=0.834) total time=   0.3s
[CV 2/3] END learning_rate=0.02; accuracy: (train=0.985, test=0.635) f1: (train=0.986, test=0.582) roc_auc: (train=0.999, test=0.733) total time=   0.2s
[CV 3/3] END learning_rate=0.02; accuracy: (train=0.995, test=0.731) f1: (train=0.995, test=0.780) roc_auc: (train=1.000, test=0.772) total time=   0.3s
[CV 1/3] END learning